In [1]:
# precs, recall (stem)
# rougle, meteor, bleu 4, EM, 

In [3]:
import os
import numpy as np
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import  Cider
import bleu

def calc_prec(pred, tgt):
    prds = pred.split()
    tgs = tgt.split()
    count = 0 
    for prd in prds:
        if prd in tgs:
            count += 1
    return count/ len(prds)

def cal_metrics(dir='output', type='test'):
    if type == 'val': idx = 0
    else: idx = 1

    pred_path = f'{dir}/test_{idx}.output'
    target_path = f'{dir}/test_{idx}.gold'
    preds = list()
    targets = list()
    
    with open(pred_path) as f:
        preds =  [line.strip() for line in f.readlines()]
    with open(target_path) as f:
        targets =  [line.strip() for line in f.readlines()]

    tmp_file= 'tmpgold.txt'
    with open(tmp_file,'w+') as f:
        f.write('\n'.join(targets))
    
    EM = list()
    precs = list()
    for i, (ref, gold) in enumerate(zip(preds, targets)):
        EM.append(ref.split() == gold.split())
        precs.append(calc_prec(ref,gold))
    (goldMap, predictionMap) = bleu.computeMaps(
        preds, tmp_file)
    dev_bleu = round(bleu.bleuFromMaps(goldMap, predictionMap)[0], 3)
    (goldMap2, predictionMap2) = bleu.computeMaps(
        preds, tmp_file)
    dev_bleu2 = round(bleu.bleuFromMaps(goldMap2, predictionMap2)[0], 3)
    EM = round(np.mean(EM)*100, 3)
    precs = round(np.mean(precs)*100, 3)
    res = {k: [' '.join(v.split('\t')[1:]).strip().lower()] for k, v in enumerate(preds)}
    tgt = {k: [' '.join(v.split('\t')[1:]).strip().lower()] for k, v in enumerate(targets)}
    
    # precision 1-gram
    print(len(res))
    print(len(tgt))
    score_Meteor, scores_Meteor = Meteor().compute_score(tgt, res)
    print("Meteor: %s" % (float(score_Meteor)*100))
    score_Rouge, scores_Rouge = Rouge().compute_score(tgt, res)
    print("ROUGE-L: %s" % (float(score_Rouge)*100))
    print(" %s = %s " % ("EM", str(EM)))
    print(" %s = %s " % ("precs", str(precs)))
    print("  %s = %s " % ("bleu-4", str(dev_bleu)))
    print("  %s = %s " % ("bleu-normal", str(dev_bleu2)))
    os.remove(tmp_file)

cal_metrics(dir='output', type='test')

Total: 6406
Total: 6406


6406
6406
Meteor: 9.218705490095015
ROUGE-L: 12.562070174206264
 EM = 0.343 
 precs = 30.562 
  bleu-4 = 13.452 
  bleu-normal = 13.452 


In [ ]:
# recall presc

In [ ]:
cache_stem = dict()
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
import pandas as pd 
with open(f'{output_dir}/test_1.output') as f:
    out = [l.strip() for l in f.readlines()]
with open(f'{output_dir}/test_1.gold') as f:
    gold = [l.strip() for l in f.readlines()]

In [ ]:
def preprocess(sentence):
    sentence = sentence.lower()
    sentence = [ps.stem(l) for l in sentence.split()]
    # print(sentence)
    return sentence
def calc(ref,pred):
    ref = preprocess(ref)
    pred = preprocess(pred)
    inter = set(ref).intersection(set(pred))
    recall = len(inter) / len(ref)
    precs = len(inter) / len(pred)
    # doan dung / tong so dung recal
    # doan dung / tong so doan prec
    return recall, precs
# read 

In [ ]:
df = pd.DataFrame()
df['out'] = out 
df['ref'] = gold

In [ ]:
recalls = list()
precisions = list()
for _,row in df.iterrows():
    r,p = calc(row['ref'],row['out'])
    recalls.append(r)
    precisions.append(p)
df['recall'] = recalls
df['precision'] = precisions

In [ ]:
import json
codes = list()
file_old = list()
file_new = list()
cms = list()
repos = list()
with open('output/versions/seq2seq_double_transformer/test.jsonl') as f:
    for l in f.readlines():
        data = json.loads(l.strip())
        code = data['code_tokens']
        msg = data['docstring_tokens']
        cms.append(data['index'])
        codes.append(' '.join(code))
len(codes)

5100

In [ ]:
df['code'] = codes
df['commit_id'] = cms
# df['repo'] = repos

ValueError: Length of values (5100) does not match length of index (3253)

In [ ]:
df = df.sort_values(by=['recall'], ascending=False)
df.to_excel('output/result.xlsx')

In [ ]:
sum(df['recall'])/ df.shape[0], sum(df['precision'])/ df.shape[0]

(0.24714737658854655, 0.3177320408349927)